# Saliency using Gradient Backpropagation

Implement the saliency algorithm described in https://arxiv.org/pdf/1312.6034.pdf.

In [1]:
REPETITIONS = 50
NOISE = .2

## Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

! cp -r '/content/drive/MyDrive/Colab Notebooks/cs-no/6.grads/config' .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
from config import config, data, model, utils

from config.utils import to_image, normalize

utils.setup_clean_image_plotting()

In [4]:
from tensorflow.python.keras.applications import mobilenet_v2

config.model.architecture = tf.keras.applications.MobileNetV2
config.model.module = mobilenet_v2
config.model.decode = tf.keras.applications.mobilenet_v2.decode_predictions
config.model.preprocess = tf.keras.applications.mobilenet_v2.preprocess_input
config.model.decompress = lambda x: (x + 1.) * 127.5

config.data.image_size = (224, 224)
config.data.input_shape = (config.data.batch_size, 224, 224, 3)

In [5]:
images = data.load_images()
x = model.preprocess(images)

In [6]:
nn = model.build()

logits = nn(x, training=False)
preds = tf.argmax(logits, axis=1)
probs = tf.nn.softmax(logits)
decoded = model.decode(probs.numpy(), top=2)

## SmoothGrad Backpropagation

In [7]:
def activation_loss(y, units):
    return tf.gather(y, units, axis=1, batch_dims=1)

@tf.function
def gradients(inputs, units):
    with tf.GradientTape(watch_accessed_variables=True) as tape:
        tape.watch(inputs)
        y = nn(inputs)
        loss = activation_loss(y, units)
    
    grads = tape.gradient(loss, inputs, unconnected_gradients=tf.UnconnectedGradients.ZERO)

    return loss, grads

In [8]:
@tf.function
def smooth_gradients(inputs, units, num_samples=50, noise=.2):
    x = tf.repeat(inputs, num_samples, axis=0)
    x += tf.random.normal(x.shape, mean=0, stddev=noise/2)

    y = tf.repeat(units, num_samples)
    loss, grads = gradients(x, y)
    
    grads = tf.reshape(grads, (-1, num_samples, *grads.shape[1:]))
    return loss, tf.reduce_mean(grads, axis=1)

In [10]:
_, g = gradients(x, preds)

sa = normalize(tf.reduce_sum(tf.abs(g), axis=-1))
sp = normalize(tf.reduce_sum(tf.nn.relu(g), axis=-1))
sn = normalize(tf.reduce_sum(tf.nn.relu(-g), axis=-1))

sg = tf.concat([smooth_gradients(x[ix:ix+1], preds[ix:ix+1], REPETITIONS, NOISE)[1]
                for ix in range(len(x))],
               axis=0)

a = normalize(tf.reduce_sum(tf.abs(g), axis=-1))
p = normalize(tf.reduce_sum(tf.nn.relu(g), axis=-1))
n = normalize(tf.reduce_sum(tf.nn.relu(-g), axis=-1))

sa = normalize(tf.reduce_sum(tf.abs(sg), axis=-1))
sp = normalize(tf.reduce_sum(tf.nn.relu(sg), axis=-1))
sn = normalize(tf.reduce_sum(tf.nn.relu(-sg), axis=-1))

In [13]:
utils.plot(sum(zip(to_image(images),
                   a.numpy(),
                   sa.numpy(),
                   to_image(images * sa[..., tf.newaxis])), ()),
           titles=['original',
                   'absolute gradients',
                   'smooth abs gradients',
                   'original * smooth abs gradients'],
           rows=len(images))

Output hidden; open in https://colab.research.google.com to view.